In [1]:
from IPython import get_ipython
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  #'last', 'last_expr'

In [ ]:
from functools import partial
import tensorflow as tf
import seqio
from t5.data import preprocessors as t5_preprocessors
from praxis import base_input, base_hyperparams
from paxml import seqio_input

In [ ]:
d = {'text': [[1, 2, 3], [2, 4]], 'label': [0, 1]}
ds = Dataset.from_dict(d)

In [ ]:
dataset_name = "rotten_tomatoes"
dataset = load_dataset(dataset_name, split="train")

In [67]:
for i, example in enumerate(dataset):
    if i == 3: break
    print(example)

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
{'text': 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'label': 1}
{'text': 'effective but too-tepid biopic', 'label': 1}


In [ ]:
cache_dir = '/nas/xd/.cache/torch/transformers/'
model_name = 'EleutherAI/gpt-j-6B'
tokenizer = GPT2Tokenizer.from_pretrained(model_name, cache_dir=cache_dir)

In [ ]:
# def tokenize_function(examples): return tokenizer(example['text'])
tokenized_dataset = dataset.map(lambda example: tokenizer(example['text']),
    batched=True, num_proc=None, remove_columns=dataset.column_names)  # run_clm_flax.py

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns('attention_mask')

In [69]:
for i, example in enumerate(tokenized_dataset):
    if i == 3: break
    print(example)

{'input_ids': [1169, 3881, 318, 23985, 284, 307, 262, 2310, 301, 4289, 338, 649, 366, 369, 272, 366, 290, 326, 339, 338, 1016, 284, 787, 257, 22870, 772, 3744, 621, 610, 77, 727, 5513, 5767, 89, 44028, 837, 474, 11025, 12, 565, 3885, 5719, 1801, 1326, 393, 2876, 574, 384, 13528, 764]}
{'input_ids': [1169, 17177, 3481, 15962, 24659, 286, 366, 262, 15876, 286, 262, 13917, 366, 26298, 318, 523, 3236, 326, 257, 5721, 286, 2456, 2314, 22668, 6901, 763, 12, 16002, 14, 35248, 279, 2357, 14509, 1559, 338, 9902, 5761, 286, 474, 764, 374, 764, 374, 764, 284, 75, 74, 2013, 338, 3504, 12, 16442, 764]}
{'input_ids': [16803, 475, 1165, 12, 83, 538, 312, 3182, 16603]}


In [ ]:
# mesh-transformer-jax, https://www.tensorflow.org/tutorials/load_data/tfrecord
def _int64_feature(value): return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def write_tfrecords(dataset, fp):
    with tf.io.TFRecordWriter(fp) as writer:
        for example in dataset:
            feature = {"input_ids": _int64_feature(example['input_ids'])}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [ ]:
fp = f'{dataset_name}_train_{len(tokenized_dataset)}.tfrecords'
write_tfrecords(tokenized_dataset, fp)

## load

In [110]:
feature_desc = {"input_ids": tf.io.VarLenFeature(tf.int64)}
def _parse_function(example_proto): # https://zhuanlan.zhihu.com/p/552951305
    return tf.io.parse_single_example(example_proto, feature_desc)
#     for name in list(example.keys()):
#         t = example[name]
#         if t.dtype == tf.int64: t = tf.cast(t, dtype=tf.int32)
#         example[name] = tf.sparse.to_dense(t, default_value=0)
        # example[name] = tf.sparse.to_dense(tf.sparse.reorder(t)) # mesh-transformer-jax
#     return example

def shard(data, batch_size=None):
    return jax.tree_map(lambda x: x.numpy().reshape(batch_size + x.shape[1:]), data)  # mtj
    
def prefetch(dataset, n_prefetch=None):
    # Taken from: https://github.com/google-research/vision_transformer/blob/master/vit_jax/input_pipeline.py
    ds_iter = iter(dataset)
    ds_iter = map(lambda x: jax.tree_map(lambda t: np.asarray(memoryview(t)), x), ds_iter)
    if n_prefetch: ds_iter = flax.jax_utils.prefetch_to_device(ds_iter, n_prefetch)
    return ds_iter

In [98]:
fp = '/nas/xd/projects/mesh-transformer-jax/rotten_tomatoes_train_8530.tfrecords'
fp = 'gs://jax_projects/data/rotten_tomatoes_train_8530.tfrecords'  # v3-8
fp = 'gs://llm_projects/data/rotten_tomatoes_train_8530.tfrecords'  # pod
ds = tf.data.TFRecordDataset(fp)
# ds = ds.shuffle(buffer_size=min(1000, len(sequences))) # flaxmodels, https://zhuanlan.zhihu.com/p/552951305
ds = ds.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.map(lambda d: {k: tf.cast(tf.sparse.to_dense(v, default_value=0), dtype=tf.int32) for k, v in d.items()})

In [97]:
for i, example in enumerate(ds):
    if i == 3: break
    print(example)

{'input_ids': <tf.Tensor: shape=(50,), dtype=int32, numpy=
array([ 1169,  3881,   318, 23985,   284,   307,   262,  2310,   301,
        4289,   338,   649,   366,   369,   272,   366,   290,   326,
         339,   338,  1016,   284,   787,   257, 22870,   772,  3744,
         621,   610,    77,   727,  5513,  5767,    89, 44028,   837,
         474, 11025,    12,   565,  3885,  5719,  1801,  1326,   393,
        2876,   574,   384, 13528,   764], dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(53,), dtype=int32, numpy=
array([ 1169, 17177,  3481, 15962, 24659,   286,   366,   262, 15876,
         286,   262, 13917,   366, 26298,   318,   523,  3236,   326,
         257,  5721,   286,  2456,  2314, 22668,  6901,   763,    12,
       16002,    14, 35248,   279,  2357, 14509,  1559,   338,  9902,
        5761,   286,   474,   764,   374,   764,   374,   764,   284,
          75,    74,  2013,   338,  3504,    12, 16442,   764],
      dtype=int32)>}
{'input_ids': <tf.Tensor: shape=(9,), d

In [108]:
# {'input_ids': TensorSpec(shape=(None,), dtype=tf.int32, name=None)}
output_features = {k.replace('input_ids', 'targets'): seqio.Feature(vocabulary=None, dtype=tspec.dtype, rank=tspec.shape.rank)
                   for k, tspec in ds.element_spec.items()}
max_len = 80  # max(len(s) for s in sequences) == 78a

In [73]:
gradient_accumulation_steps = 16
# train_mbs_per_replica = 2 # train_micro_batch_size_per_gpu in deepspeed
tpu_size = 8  # jax.num_devices()
cores_per_replica = 4  # mp=4, dp=8/4=2
per_replica_batch = 1  # train_mbs_per_replica in run_clm_mp_xd.py, train_micro_batch_size_per_gpu in deepspeed
train_batch_size = (gradient_accumulation_steps, per_replica_batch * tpu_size // cores_per_replica)

# ds = ds.apply(tf.data.experimental.dense_to_ragged_batch(np.prod(self.bs), drop_remainder=True)) # mtj
ds = ds.padded_batch(batch_size=np.prod(train_batch_size), padded_shapes={'input_ids': [max_len]},
                     padding_values={'input_ids': 0}, drop_remainder=True)

In [74]:
for batch in ds: print(batch); break

{'input_ids': <tf.Tensor: shape=(32, 80), dtype=int32, numpy=
array([[ 1169,  3881,   318, ...,     0,     0,     0],
       [ 1169, 17177,  3481, ...,     0,     0,     0],
       [16803,   475,  1165, ...,     0,     0,     0],
       ...,
       [  270,  5419,   326, ...,     0,     0,     0],
       [ 5162,  4741,  2308, ...,     0,     0,     0],
       [   64,  4958,   913, ...,     0,     0,     0]], dtype=int32)>}


In [75]:
ds = ds.prefetch(10)  # mesh-transformer-jax
# ds = ds.repeat()  # gpt-neo/inputs.py
# map shard directly over ds won't work, getting AttributeError: 'Tensor' object has no attribute 'numpy'
# because inside tf.function?, see e.g.:
# 1) https://stackoverflow.com/questions/34097281/convert-a-tensor-to-numpy-array-in-tensorflow
# 2) https://github.com/tensorflow/tensorflow/issues/27519
# ds = ds.map(partial(shard, batch_size=train_batch_size), num_parallel_calls=tf.data.AUTOTUNE)
# matthias-wright/flaxmodels/training/stylegan2/data_pipeline.py
ds_iter = iter(ds)
ds_iter = map(lambda x: shard(x, batch_size=train_batch_size), ds_iter)

In [76]:
for batch in ds_iter: print(batch); break

{'input_ids': array([[[ 1169,  3881,   318, ...,     0,     0,     0],
        [ 1169, 17177,  3481, ...,     0,     0,     0]],

       [[16803,   475,  1165, ...,     0,     0,     0],
        [  361,   345,  3360, ...,     0,     0,     0]],

       [[24677,  3212,   355, ...,     0,     0,     0],
        [ 1169,  2646,  3769, ...,     0,     0,     0]],

       ...,

       [[ 2777,  1304,   805, ...,     0,     0,     0],
        [  272,  7306,  2569, ...,     0,     0,     0]],

       [[  265,   546,  6957, ...,     0,     0,     0],
        [  270,  5419,   326, ...,     0,     0,     0]],

       [[ 5162,  4741,  2308, ...,     0,     0,     0],
        [   64,  4958,   913, ...,     0,     0,     0]]], dtype=int32)}


In [77]:
batch['input_ids'].shape

(16, 2, 80)

In [ ]:
@seqio.map_over_dataset
def convert_datatype(ex): return {k: tf.cast(tf.sparse.to_dense(v, default_value=0), dtype=tf.int32) for k, v in ex.items()}

task_feature_lengths = {'targets': max_len}

task_name = "rotten_tomatoes_train_8530"
# seqio.TaskRegistry.add(task_name, 
task = seqio.Task(task_name, 
    seqio.TFExampleDataSource(split_to_filepattern={'train': fp}, feature_description=feature_desc),
    preprocessors=[
        convert_datatype,
        # from paxml.c4 TaskRegistry.add_versioned_tfds_task('c4_lm_v301_gpt', ...
        partial(t5_preprocessors.rekey, key_map={'targets': 'input_ids'}),
    ],
    output_features=output_features,
)

# dataset = seqio.get_mixture_or_task(task_name).get_dataset(sequence_length=task_feature_lengths, split="train")
dataset = task.get_dataset(sequence_length=task_feature_lengths, split="train", shuffle=False)

In [ ]:
p = seqio_input.SeqIOInput.HParams(
    mixture_name='super_glue_wsc_v102_simple_train',
    split_name='train',
    task_feature_lengths={'targets': 1280},
    feature_converter=seqio_input.LanguageModelFeatures(pack=True),
    is_training=True,
    use_cached=False,
    input_random_seed=123,
    batch_size=4)

is_training = True
p = seqio_input.SeqIOInput.HParams(
    name='rtTrain' if is_training else 'rtValid',
#     mixture_name= "rotten_tomatoes_train_8530" if is_training else "rotten_tomatoes_dev_100",
    mixture_or_task=task,
    split_name='train' if is_training else 'valid',
    task_feature_lengths={'targets': 128},
    use_cached=False,
    repeat=True if is_training else False,
    feature_converter=seqio_input.LanguageModelFeatures(
        pack=True if is_training else False,
#             use_custom_packing_ops=False,
            bos_id=0,
#             reverse_bos_padding=True, eos_id=0,
    ),
    is_training=is_training,
    input_random_seed=(42 if is_training else 4321),
    batch_size=2,  # TODO
    drop_remainder=True if is_training else False,
#         num_batches_to_skip=self.TRAINING_NUM_BATCHES_TO_SKIP,
    num_infeed_hosts=1,  # TODO
    reset_for_eval=False if is_training else True,
    annotate_padding_fields=True,
    shuffle=False,  # debug
)

In [ ]:
p0 = p.clone().set(infeed_host_index=0, num_infeed_hosts=2)
p1 = p.clone().set(infeed_host_index=1, num_infeed_hosts=2)
inp0 = base_hyperparams.instantiate(p0)
inp1 = base_hyperparams.instantiate(p1)

batch = inp0.get_next()
for k, v in batch.FlattenItems():
  print(k, v.shape, v.dtype)

In [ ]:
pip install tensorflow==2.9.2
pip install tensorflow-text==2.9.0

pip install ipython

git clone https://github.com/xiaoda99/seqio.git
git clone https://github.com/xiaoda99/praxis.git
git clone https://github.com/xiaoda99/paxml.git
# from paxml: orbax-checkpoint 0.2.3 has requirement jax>=0.4.9
git clone https://github.com/xiaoda99/praxis.git projects/praxis; cd projects/praxis; git checkout xd/dev; /home/lishengping/miniconda3/bin/pip install -e .
git clone https://github.com/xiaoda99/paxml.git projects/paxml; cd projects/paxml; git checkout xd/dev; /home/lishengping/miniconda3/bin/pip install -e .

gcloud compute tpus tpu-vm scp /nas/xd/projects/paxml/paxml/tasks/lm/model_params.py llm-jax-v3-32:/home/lishengping/projects/paxml/paxml/tasks/lm/ --zone=us-east1-d  --project llm-tpu --worker all
gcloud compute tpus tpu-vm scp /nas/xd/projects/paxml/paxml/tasks/lm/params/c4.py llm-jax-v3-32:/home/lishengping/projects/paxml/paxml/tasks/lm/params/ --zone=us-east1-d  --project llm-tpu --worker all
gcloud compute tpus tpu-vm ssh llm-jax-v3-32 --zone=us-east1-d  --project llm-tpu --worker all --command="/home/lishengping/miniconda3/bin/python /home/lishengping/projects/paxml/paxml/main.py --exp=tasks.lm.params.c4.C4Spmd2BAdam32Replicas --job_log_dir=gs://llm_projects/log/C4Spmd2BAdam32Replicas 2>&1 | tee train.log"

In [ ]:
mesh_shape = (16, 2)
devices = np.asarray(jax.devices()).reshape(*mesh_shape)
mesh = Mesh(devices, ('x', 'y'))

if jax.process_index() == 0:
 input_data = np.arange(0,16).reshape(8,2)
elif jax.process_index() == 1:
 input_data = np.arange(16,32).reshape(8,2)
elif jax.process_index() == 2:
 input_data = np.arange(32,48).reshape(8,2)
else:
 input_data = np.arange(48,64).reshape(8,2)
rank = jax.process_index()
input_data = np.arange(rank * 16, (rank + 1) * 16).reshape(8,2); input_data
f = pjit(lambda x: x, in_axis_resources=P(('x', 'y'),), out_axis_resources=P('x', 'y'))
with mesh: data = f(input_data)
with mesh:
    multihost_utils.sync_global_devices('aaa')
    data = f(input_data)

In [ ]:
import numpy as np

import jax
from jax import pmap
from jax.experimental.maps import Mesh
from jax.experimental.pjit import pjit
from jax.experimental import PartitionSpec as P
mesh_shape = (16, 2)
devices = np.asarray(jax.devices()).reshape(*mesh_shape)
mesh = Mesh(devices, ('x', 'y'))
rank = jax.process_index()
input_data = np.arange(rank * 16, (rank + 1) * 16).reshape(8,2)
rank
rank = jax.process_index()
input_data = np.arange(rank * 16, (rank + 1) * 16).reshape(8,2); input_data
f = pjit(lambda x: x, in_axis_resources=P(('x', 'y'),), out_axis_resources=P('x', 'y'))
with mesh: data = f(input_data)
with Mesh(mesh.devices, mesh.axis_names): data = f(input_data)
from jax.experimental import multihost_utils
multihost_utils.sync_global_devices('aaa')
with mesh:
    multihost_utils.sync_global_devices('aaa')
    data = f(input_data)
